In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

In [2]:
import scipy.ndimage as ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import build_montages
import matplotlib.pyplot as plt
import argparse
import cv2
from keras.utils import np_utils
from imutils.contours import sort_contours
import imutils

In [3]:
from google.colab.patches import cv2_imshow

In [4]:
num_classes = 75
img_size = 60

In [5]:
dataset=pd.read_csv("/content/drive/MyDrive/emnist/newdataset-60-62-x2.csv").astype('float32')

In [13]:
dataset=pd.read_csv("/content/drive/MyDrive/emnist/newdataset-60-75-x2-x4ne.csv").astype('float32')

In [14]:
newdataset=np.array(dataset)

In [15]:
# chia 60% train set, 40% cho validation set và test set
split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, test_index in split.split(newdataset, newdataset[:,0]):
  strat_train_set = newdataset[train_index]
  strat_val_test_set = newdataset[test_index]

In [16]:
#từ 40% validation-test set, chia 20% cho test set và 20% cho validation set
split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for test_index, val_index in split2.split(strat_val_test_set, strat_val_test_set[:,0]):
  strat_test_set = strat_val_test_set[test_index]
  strat_val_set = strat_val_test_set[val_index]

In [17]:
#chuyển dataset từ dataframe sang numpy arry và rescale thông số từ 0-255 sang 0-1
X = np.array(strat_train_set[:,1:])/255.
#chuyển dataset từ array có shape (3600,1) sang ma trận (60,60)
X_train=X.reshape(X.shape[0],img_size,img_size,1)
y = strat_train_set[:,0]
# chuyển labels sang one-hot encoding
labels = keras.utils.to_categorical(y, num_classes)
#tương tự với validation set
X_val=  np.array(strat_val_set[:,1:])/255
X_val=X_val.reshape(X_val.shape[0],img_size,img_size,1)
y_val = strat_val_set[:,0]
labels_val=keras.utils.to_categorical(y_val, num_classes)
#tương tự với test set
X_test=  np.array(strat_test_set[:,1:])/255
X_test=X_test.reshape(X_test.shape[0],img_size,img_size,1)
y_test = strat_test_set[:,0]
labels_test=keras.utils.to_categorical(y_test, num_classes)

In [18]:
checkpoint_filepath ="/content/drive/MyDrive/checkpoint-model3/bestmodel2-9.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(img_size, img_size, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'],run_eagerly=True)
history = model.fit(X_train, labels, validation_data=(X_val, labels_val), epochs=100, batch_size=200, verbose=2,callbacks=[model_checkpoint_callback])



In [22]:
model=keras.models.load_model("/content/drive/MyDrive/checkpoint-model3/bestmodel2-9.hdf5")

In [ ]:
preds = model.evaluate(X_test, labels_test)